In [8]:
import os
import numpy as np

In [9]:
# Constants
OMEGA_BARYON = 0.046026
C = 300000 # km/s

# file parts here

# for parameter file
param_part = '../../file_parts/param_811.part'
hydro_part = '../../file_parts/hydro.part'
SIDM_part = '../../file_parts/SIDM.part'

# for submission script
doit_part = '../../file_parts/doit.part'

# for Config.sh
config_part = '../../file_parts/Config.part'
config_DM_part = '../../file_parts/Config-DM.part'
config_HY_part = '../../file_parts/Config-Hydro.part'
config_2cDM_part = '../../file_parts/Config-2cDM.part'
config_Double_part = '../../file_parts/Config-DoublePrecision.part'

# output directory
arepo_out = '../../param/arepo/811/'
base_out = '../../param/'

In [10]:
# Simulation Parameters
# Change these
# code_directory = 0

# N_part = 256
# boxsize = 25 # in Mpc

nodes = 4
mem_size = 8 # in GB

DM_type = '2cDM'
hydro_type = 'HY'
power = 'power00'
sigma=1
extra_info = ''
# V_k = 100

heavy_fraction = 0.5

#ic_file = f'L{boxsize}N{N_part}'
#ic_file = f'IC-L{boxsize}N{N_part}-{code_directory}'
double_input = True

dry_run = False

In [11]:
    #sigmas = [ f'{x:.3}' for x in np.geomspace(0.5, 5, num=10)]
V_ks = np.geomspace(20, 200, num=10)

    #N_dirs = 10
for code_directory, V_k in enumerate(V_ks):

    directory = f'dir_{code_directory}'

    cross_section = f'{power}_sigma{sigma}'
    extra_info = f'_Vkick{V_k:.2f}'

    with open(param_part, 'r') as f:
        param_text = f.read()

    with open(doit_part, 'r') as f:
        doit_text = f.read()

    with open(config_part, 'r') as f:
        config_text = f.read()

    if DM_type == 'CDM':
        #job_string = DM_type + f'_L{boxsize}N{N_part}_' + hydro_type + '_' + directory + extra_info
        job_string = DM_type + f'_811_' + hydro_type + '_' + directory + extra_info
    else:
        #job_string = DM_type + f'_L{boxsize}N{N_part}_' + hydro_type + '_' + cross_section + '_' + directory + extra_info
        job_string = DM_type + f'_811_' + hydro_type + '_' + cross_section + '_' + directory + extra_info
    # boxsize_kpc = boxsize * 1000
    # softening_length = boxsize_kpc / (29 * N_part )
    mem_size_MB = mem_size * 1000 

    # creating output directory
    param_name = job_string + '.txt'
    out_dir = base_out + f'{job_string}/'

    try:
        os.mkdir(out_dir)
    except:
        print(f'{out_dir} already exists!')

    # replacing text in parameter file
    # param_text = param_text.replace('{$IC-FILE}', ic_file)
    param_text = param_text.replace('{$JOBSTRING}', job_string)
    param_text = param_text.replace('{$MEMSIZE}', str(mem_size_MB))
    param_text = param_text.replace('{$CODE_DIR}', str(code_directory))
    # param_text = param_text.replace('{$BOXSIZE}', str(boxsize_kpc))

    # param_text = param_text.replace('{$SOFTENING_LENGTH}', str(softening_length))
    # param_text = param_text.replace('{$SOFTENING_LENGTH_HALF}', str(softening_length / 2)) 

    # replacing text in config file
    # N_eff = 256 if N_part <= 256 else N_part

    N_eff = 512
    config_text = config_text.replace('{$PMGRID_SIZE}', str(N_eff))

    if double_input:
        with open(config_Double_part, 'r') as f:
            config_text += f.read()

    if DM_type != 'CDM':
        with open(SIDM_part, 'r') as f:
            param_text += f.read()
    if DM_type == '2cDM':
        with open(config_2cDM_part, 'r') as f:
            config_text += f.read()

    if hydro_type == 'HY':
        hydro_flag = 1
        omega_baryon = OMEGA_BARYON
        with open(hydro_part, 'r') as f:
            param_text += f.read()
        with open(config_HY_part, 'r') as f:
            config_text += f.read()
    else:
        hydro_flag = 0
        omega_baryon = 0
        with open(config_DM_part, 'r') as f:
            config_text += f.read()
    param_text = param_text.replace('{$OMEGA_BARYON}', str(omega_baryon))
    param_text = param_text.replace('{$HY_FLAG}', str(hydro_flag))

    # replacing text in doit script
    doit_text = doit_text.replace('{__MEMSIZE__}', f'{mem_size}g')
    doit_text = doit_text.replace('{__JOBSTRING__}', job_string)
    doit_text = doit_text.replace('{__CODE_DIR__}', str(code_directory))
    doit_text = doit_text.replace('{__NODES__}', str(nodes))
    doit_text = doit_text.replace('{__PARAM_FILE__}', param_name)
    doit_restart_text = doit_text.replace('{__RESTART__}', str(1))
    doit_text = doit_text.replace('{__RESTART__}', str(0))

    if DM_type == '2cDM':
        delm = 1/2 * (V_k)**2 / C**2

        scatter_states_text = f'0.0 {1 - heavy_fraction}\n{delm:.3} {heavy_fraction}\n'
        scatter_states_out = out_dir + 'scatter_states.txt'
        
        if not dry_run:
            with open(scatter_states_out, 'w') as f:
                f.write(scatter_states_text)

    param_out = out_dir + param_name
    param_out1 = arepo_out + param_name
    doit_out = out_dir + 'doit.sh'
    doit_restart_out = out_dir + 'doit-restart.sh'
    config_out = out_dir + 'Config.sh'

    if not dry_run:
        with open(param_out, 'w') as f:
            f.write(param_text)
        with open(param_out1, 'w') as f:
            f.write(param_text)
        with open(doit_out, 'w') as f:
            f.write(doit_text)
        with open(doit_restart_out, 'w') as f:
            f.write(doit_restart_text)
        with open(config_out, 'w') as f:
            f.write(config_text)

../../param/2cDM_811_HY_power00_sigma1_dir_0_Vkick20.00/ already exists!
../../param/2cDM_811_HY_power00_sigma1_dir_1_Vkick25.83/ already exists!
../../param/2cDM_811_HY_power00_sigma1_dir_2_Vkick33.36/ already exists!
../../param/2cDM_811_HY_power00_sigma1_dir_3_Vkick43.09/ already exists!
../../param/2cDM_811_HY_power00_sigma1_dir_4_Vkick55.65/ already exists!
../../param/2cDM_811_HY_power00_sigma1_dir_5_Vkick71.88/ already exists!
../../param/2cDM_811_HY_power00_sigma1_dir_6_Vkick92.83/ already exists!
../../param/2cDM_811_HY_power00_sigma1_dir_7_Vkick119.90/ already exists!
../../param/2cDM_811_HY_power00_sigma1_dir_8_Vkick154.85/ already exists!
../../param/2cDM_811_HY_power00_sigma1_dir_9_Vkick200.00/ already exists!


In [12]:
if dry_run:
    print(param_text)

In [13]:
if dry_run:
    print(doit_text)

In [14]:
if dry_run:
    print(config_text)